# Spot the Mask Challenge:

Can you predict whether a person in an image is wearing a face mask?

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import os
import shutil

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.layers import Input, Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import utils # My Custom functions

### Preparing the data:
> Phase1:
* Load the images into the workspace
* Split the images into train and test images
* Create train and test image directories
* Move images to the train and test directories.

In [ ]:
# Data loading
image_dir = "./images/"
train_dir = "./raw/train_labels.csv"
sub_dir = "./raw/sample_sub.csv"

sub = pd.read_csv(sub_dir)
train_data = pd.read_csv(train_dir)

In [ ]:
# Global variables
FAST_RUN = False
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
# Set seed for reproducibility
tf.random.set_seed(42)

###  Creating train and test directories to store the respective images:
Note: These methods are to be run once:

In [ ]:
# Invoking the create_dir method in utils.py ^_^
dirs = ['train', 'test']
utils.create_dir(dirs, image_dir)

In [ ]:
# Split dataset into train and test images
img_names = os.listdir(image_dir)
train_img_names = train_data.image.tolist()
test_img_names = []

for img in img_names:
    if img not in train_img_names:
        test_img_names.append(img)

# Move train images to the train dir
utils.move_images(train_img_names, image_dir, './images/train')

# Move test images to the test dir
utils.move_images(test_img_names, image_dir, './images/test')

### Preparing the data:
> Phase2:
* Analyse the spread of images between the classes to check whether the dataset is balanced or not.
* Split the train images further into train and validation images.
* Create train and validation image generators.
 * I'll be using tensorflow's (keras) `ImageDataGenerator()` to load the images. The Dataset API `(tf.data.Dataset())` provides another way of loading data with tensorflow.
 * Image transformation such as normalizing pixel values and augmentations such as horizontal flipping.

In [ ]:
# Plotting the image class distributions in the train set
train_data['target'].value_counts().plot.bar()
plt.title('Image class distributions')
plt.xlabel('Classes (0: No mask) (1: Mask)')
plt.ylabel('Number of images in train set')

When using the image generator with `class_mode="categorical"`, the `target` column needs to be a string: The image generator will then one-hot encode it.
So I'll create a mapping `{0: 'No_mask', 1: 'Mask'}` of the target classes from integer to string ; given that there are only two classes represented in the dataset.i.e images with masks (or with people wearing masks) and those without masks in them.

And finally split the train set into a train and validation set.

In [ ]:
# Further splitting the train images into train and validation images.
train_data['target'] = train_data['target'].replace({0: 'No_mask', 1: 'Mask'})

tr_data, val_data = train_test_split(train_data, test_size=0.20, random_state=42)
tr_data = tr_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

In [ ]:
# Re-check the the image class distributions
tr_data['target'].value_counts().plot.bar()

In [ ]:
total_train = tr_data.shape[0]  #total train images
total_validate = val_data.shape[0] #total images on validation set
batch_size = 8

In [ ]:
# Train generator
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    tr_data, 
    './images/train/train/', 
    x_col = 'image',
    y_col = 'target',
    target_size = IMAGE_SIZE,
    class_mode = 'categorical',
    batch_size = batch_size
)

# Validation generator
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    val_data, 
    './images/train/train/', 
    x_col='image',
    y_col='target',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(0, 8):
    plt.subplot(2, 4, i+1)
    for X_batch, Y_batch in train_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

### Training:
Approach-1 (Build ConvNet from scratch)

Task-bag:
* Define the Model and its attributes `(Model Architecture)`
    * I'll make use of the `Sequential()` api defined in `tf.keras.models`
* Compile the model
    * Since there are 2 classes of images in the dataset, I'll set `loss=binary_crossentropy`.
* Define training parameters and optimizations
    * EarlyStopping
    * learning rate decay.


In [ ]:
# Defining the model
model = Sequential()
#Layer1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#Layer2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#Layer3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

#Classification layer
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#Output layer
model.add(Dense(2, activation='softmax')) # 2 because there are 2 classes

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0001), metrics=['accuracy'])
model.summary()

In [ ]:
earlystop = EarlyStopping(patience=5) # Stop if validation loss doesn't improve after 5 epochs

# Gradually reduce the learning rate if validation loss doesn't improve after 5 steps
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001
                                           )

callbacks = [earlystop, learning_rate_reduction]


In [ ]:
# Training the model
epochs = 10 if FAST_RUN else 100 #200
history = model.fit_generator(
    train_generator, 
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = 70, #total_validate//batch_size,
    steps_per_epoch = 70, #total_train//batch_size,
    callbacks = callbacks
)

In [ ]:
model.save_weights("model.h5")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

### Transfer Learning (Approach-2):
Given that the dataset is small `~1300 train images`, its image size is insufficient to train a model that can generalize well on unseen data.

**Transfer learning** is a technique of model training that relies on a pretrained model (model already trained on a larger dataset) to train a new one in the same domain area (Computer Vision, NLP or RL).

*Modes of training with transfer learning:*
* Feature extraction:

     * One way of doing transfer learning is by instantiating a pre-trained model without the classification/top layer, adding a custom fully-connected layer i.e a `Dense()` layer on top and `freezing` the pretrained model so that only the weights of the new model get updated during training.
     
    With this method, the convolutional base of the pretrained model acts as a `feature extractor`; extracting all features associated with each image and the `Dense()` layer (top layer) determines the image class from the set of features.
 
* Fine tuning:
    * Here the pretrained model is trained further (usually from a specific layer to the top/end) to improve performance.
   
More about transfer learning: [Here](https://www.tensorflow.org/tutorials/images/transfer_learning)

In [ ]:
# Loading the MobileNet-V2 pre-trained model
# I'll be using it as a feature extractor here
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
#Freezing Model Weights (convolutional base)
#Freezing (by setting layer.trainable = False) prevents the weights of the pretrained model from being updated during training.
mobile_net.trainable = False

# Visualizing base model architecture
mobile_net.summary()

In [ ]:
# Defining the model for training
mobileNet_model = Sequential([
    mobile_net,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(2, activation='softmax')
])

base_learning_rate = 0.0001

mobileNet_model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
mobileNet_model.summary()

In [ ]:
epochs = 10 if FAST_RUN else 200
hist = mobileNet_model.fit_generator(
    train_generator, 
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = 60,
    steps_per_epoch = 60,
    callbacks = callbacks
)

In [ ]:
# Save model weights
mobileNet_model.save_weights("mobilenet.h5")

### Predicting on the test set

In [ ]:
test_filenames = os.listdir('./images/test/test/')
test_df = pd.DataFrame({
    'image': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    './images/test/test/', 
    x_col='image',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

For categoral classication the prediction will come with probability of each category. So we will pick the category that have the highest probability with numpy average max

In [ ]:
#predictions = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))
predictions = mobileNet_model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

For categorical classication the prediction is the probability of each class being present in an image, so we will pick the class with the highest probability.

In [ ]:
np.argmax(predictions, axis=-1)

In [ ]:
test_df['target'] = predictions

In [ ]:
test_df.head()

In [ ]:
sub = test_df.copy()
sub.head()

In [ ]:
#os.mkdir('./submissions')
sub.to_csv('./submissions/submission.csv', index=False)

Notes:
* A better score using both approaches can be attained.
    * Approach-1: 
        * Do more data augmentation
        * Change model architecture.e.g. add more convolutional layers
        * Tweaking other optimization params; learning_rate, batch_size, etc
    * Approach-2:
        * Data augmentation also applies here.
        * Use other Hyper-params
